In [1]:
!pip install -r "requirements.txt"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 10.0 MB/s eta 0:00:00


In [1]:
import torch
import math
from torch import nn, dtype
import torch.nn.functional as F
from torch.optim import Optimizer
from torch.utils.data import random_split, DataLoader, Dataset
from config import PretrainedConfig, GPT2Config
from transformers import GPT2Model as OpenAIGPT2Model
from transformers import GPT2Tokenizer
from utils import *
from einops import rearrange
from typing import Callable, Iterable, Tuple
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [3]:
import os

# Check current directory
print("Current directory:", os.getcwd())
print("Files in current directory:", os.listdir('.'))

# If XNLI folders aren't visible, you might need to change directory
# Uncomment and adjust if needed:
# os.chdir('/path/to/your/project/folder')

Current directory: /notebooks
Files in current directory: ['requirements.txt', 'model_en_epoch_3.pt', 'XNLI-1.0', '.Trash-0', 'pretrain.txt', 'best_model', 'optimizer_test.npy', 'Task3_part 1.ipynb', '.git', 'utils.py', '__pycache__', 'task4.ipynb', 'toy_gpt2_model_full.pth', 'config.py', '.ipynb_checkpoints', 'XNLI-MT-1.0']


## Task 3 Fine-tune GPT-2 (per-language)

Guidance: Load the pretrained GPT-2 (not the ones fine-tuned on English NLI) along with the training data for a single target language. Choose non-English languages that performed well in the zero-shot cross-lingual transfer and fertility evaluation. It depends on you how many languages to include. Fine-tune a separate model for each selected language. Afterwards, compare these per-language fine-tuned models with the zero-shot cross-lingual transfer results.

## Tokenizer

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# text = "Welcome, this is the beginning of default final project!"
# input_ids = tokenizer(text)['input_ids']
# print('input_ids:', input_ids)
# for token in input_ids:
#     print('token', tokenizer.decode(token))

## Model

In [12]:
class CausalSelfAttention(nn.Module):
  def __init__(self, config):
    super().__init__()

    self.num_attention_heads = config.num_attention_heads
    self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
    self.all_head_size = self.num_attention_heads * self.attention_head_size

    # Initialize the linear transformation layers for key, value, query.
    self.query = nn.Linear(config.hidden_size, self.all_head_size)
    self.key = nn.Linear(config.hidden_size, self.all_head_size)
    self.value = nn.Linear(config.hidden_size, self.all_head_size)
    # This dropout is applied to normalized attention scores following the original
    # implementation of transformer. Although it is a bit unusual, we empirically
    # observe that it yields better performance.
    self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

  def transform(self, x, linear_layer):
    # The corresponding linear_layer of k, v, q are used to project the hidden_state (x).
    proj = linear_layer(x)
    # Next, we need to produce multiple heads for the proj. This is done by spliting the
    # hidden state to self.num_attention_heads, each of size self.attention_head_size.
    proj = rearrange(proj, 'b t (h d) -> b t h d', h=self.num_attention_heads)
    # By proper transpose, we have proj of size [bs, num_attention_heads, seq_len, attention_head_size].
    proj = rearrange(proj, 'b t h d -> b h t d')
    return proj

  def attention(self, key, query, value, attention_mask):
    """
    TODO-1: Compute scaled dot-product attention for GPT-2.

    Implementation hints:
    1. Compute raw attention scores using QK^T, and scale them by sqrt(d_k).
    2. Apply a causal mask (lower-triangular) to prevent attending to future tokens.
    3. Optionally add the external attention_mask (e.g., padding positions).
    4. Normalize the scores with softmax to obtain attention probabilities.
    5. Apply dropout on the probabilities.
    6. Use them to weight the values (V) and obtain the context vectors.
    7. Finally, merge all attention heads back into a single hidden representation.
    """
    ### YOUR CODE HERE
    #query=(num_queries,d_k)
    #compute raw attention scores
    attention_scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.attention_head_size)
    #apply causal mask
    #create the lower triangular matrix
    seq_len = attention_scores.size(-1)
    causal_mask = torch.tril(torch.ones((seq_len, seq_len), device=attention_scores.device))
    causal_mask = causal_mask.unsqueeze(0).unsqueeze(0) #add batch and head dimensions
    #apply mask, set positions with 0 to -inf
    attention_scores = attention_scores.masked_fill(causal_mask == 0, -1e9)
    #add external attention mask
    if attention_mask is not None:
      attention_scores = attention_scores + attention_mask
    #normalize scores with softmax
    attention_probs = torch.softmax(attention_scores, dim=-1)
    #apply dropout
    attention_probs = self.dropout(attention_probs)
    #weigh the values(attention_prob@value)
    context_vector=torch.matmul(attention_probs,value)
    #merge all attention heads into 1
    batch_size, num_heads, seq_len, head_size = context_vector.size()
    context = context_vector.transpose(1, 2).contiguous().reshape(batch_size, seq_len, num_heads * head_size)


    return context


  def forward(self, hidden_states, attention_mask):
    """
    hidden_states: [bs, seq_len, hidden_state]
    attention_mask: [bs, 1, 1, seq_len]
    output: [bs, seq_len, hidden_state]
    """
    # First, we have to generate the key, value, query for each token for multi-head attention
    # using self.transform (more details inside the function).
    # Size of *_layer is [bs, num_attention_heads, seq_len, attention_head_size].
    key_layer = self.transform(hidden_states, self.key)
    value_layer = self.transform(hidden_states, self.value)
    query_layer = self.transform(hidden_states, self.query)
    
    # Calculate the multi-head attention using the self.attention function.
    attn_value = self.attention(key_layer, query_layer, value_layer, attention_mask)
    return attn_value

In [11]:
class GPT2Layer(nn.Module):
  def __init__(self, config):
    super().__init__()
    # Multi-head attention.
    self.self_attention = CausalSelfAttention(config)
    # Add-norm for multi-head attention.
    self.attention_dense = nn.Linear(config.hidden_size, config.hidden_size)
    self.attention_layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
    self.attention_dropout = nn.Dropout(config.hidden_dropout_prob)
    # Feed forward.
    self.interm_dense = nn.Linear(config.hidden_size, config.intermediate_size)
    self.interm_af = F.gelu
    # Add-norm for feed forward.
    self.out_dense = nn.Linear(config.intermediate_size, config.hidden_size)
    self.out_layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
    self.out_dropout = nn.Dropout(config.hidden_dropout_prob)

  def add(self, input, output, dense_layer, dropout):
    """
    TODO-2: Residual connection with dense projection and dropout.
    
    Implementation hints:
    1. Project the 'output' through dense_layer.
    2. Apply dropout to prevent overfitting.
    3. Add the original 'input' (residual connection) to the processed output.
    """

    ### YOUR CODE HERE
    #project output through dense_layer
    projected=dense_layer(output)
    #apply dropout
    projected=dropout(projected)
    #add OG input to output
    return input+projected


  def forward(self, hidden_states, attention_mask):
    """
    TODO-3: Forward pass of a GPT-2 layer.

    Implementation hints:
    ---- Self-Attention Block ----
    1. LayerNorm the input for stability using self.attention_layer_norm.
    2. Compute multi-head causal self-attention using self.self_attention.
    3. Apply residual connection using self.add (dense_layer=self.attention_dense, dropout=self.attention_dropout).

    ---- Feed Forward Block ----
    4. LayerNorm the hidden_states from attention block using self.out_layer_norm.
    5. Pass through a two-layer feed-forward network with activation:
       self.interm_dense -> self.interm_af -> self.out_dense
    6. Apply residual connection again using self.add (dense_layer=self.out_dense, dropout=self.out_dropout).
    """
    ### YOUR CODE HERE
    #1. layernorm the input
    normed_input=self.attention_layer_norm(hidden_states)
    #2. compute the self attention
    attn_output=self.self_attention(normed_input,attention_mask)
    #3. apply residual connection
    hidden_states=self.add(hidden_states,attn_output,self.attention_dense,self.attention_dropout)
    #4. layernorm the hidden states from attention blocks
    normed_ff=self.out_layer_norm(hidden_states)
    #5. pass through 2 layer feed forward
    ff_output=self.interm_dense(normed_ff)
    ff_output=self.interm_af(ff_output)
    ff_output = self.out_dense(ff_output)
    ff_output = self.out_dropout(ff_output)
    #6. apply residual connection
    hidden_states = hidden_states + ff_output
    return hidden_states

In [4]:
class GPTPreTrainedModel(nn.Module):

  def __init__(self, config: PretrainedConfig, *inputs, **kwargs):
    super().__init__()
    self.config = config
    self.name_or_path = config.name_or_path

  def init_weights(self):
    # Initialize weights
    self.apply(self._init_weights)

  def _init_weights(self, module):
    """ Initialize the weights """
    if isinstance(module, (nn.Linear, nn.Embedding)):
      # Slightly different from the TF version which uses truncated_normal for initialization
      # cf https://github.com/pytorch/pytorch/pull/5617
      module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
    elif isinstance(module, nn.LayerNorm):
      module.bias.data.zero_()
      module.weight.data.fill_(1.0)
    if isinstance(module, nn.Linear) and module.bias is not None:
      module.bias.data.zero_()

  @property
  def dtype(self) -> dtype:
    return get_parameter_dtype(self)

In [5]:
class GPT2Model(GPTPreTrainedModel):
  """
  The GPT model returns the final embeddings for each token in a sentence.

  The model consists of:
  1. Embedding layers (used in self.embed).
  2. A stack of n GPT layers (used in self.encode).
  3. A linear transformation layer for the [CLS] token (used in self.forward, as given).
  """

  def __init__(self, config):
    super().__init__(config)
    self.config = config

    # Embedding layers.
    self.word_embedding = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
    self.pos_embedding = nn.Embedding(config.max_position_embeddings, config.hidden_size)
    self.embed_dropout = nn.Dropout(config.hidden_dropout_prob)

    # Register position_ids (1, len position emb) to buffer because it is a constant.
    position_ids = torch.arange(config.max_position_embeddings).unsqueeze(0)
    self.register_buffer('position_ids', position_ids)

    # GPT-2 layers.
    self.gpt_layers = nn.ModuleList([GPT2Layer(config) for _ in range(config.num_hidden_layers)])

    # [CLS] token transformations.
    self.pooler_dense = nn.Linear(config.hidden_size, config.hidden_size)
    self.pooler_af = nn.Tanh()

    # Final layer norm.
    self.final_layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

    self.init_weights()

  def embed(self, input_ids):
    """
    TODO-4: Embedding layer of the GPT-2 model.

    Implementation hints:
    1. Use self.word_embedding to convert input_ids to embeddings.
    2. Generate position ids and convert to embeddings using self.pos_embedding.
    3. Sum token and position embeddings.
    4. Apply self.embed_dropout to the sum.
    """
    ### YOUR CODE HERE
    #get token embeddings
    token_embeddings=self.word_embedding(input_ids)
    batch_size, seq_len = input_ids.size()
    #generate position ids and convert to embeddings
    position_ids = self.position_ids[:, :seq_len]
    position_ids = position_ids.expand(batch_size, seq_len)
    position_embeddings = self.pos_embedding(position_ids)
    #sum token and position embeddings
    sum_of_embeddings=token_embeddings+position_embeddings
    sum_of_embeddings=self.embed_dropout(sum_of_embeddings)
    return sum_of_embeddings

  def encode(self, hidden_states, attention_mask):
    """
    hidden_states: the output from the embedding layer [batch_size, seq_len, hidden_size]
    attention_mask: [batch_size, seq_len]
    """
    # Get the extended attention mask for self-attention.
    # Returns extended_attention_mask of size [batch_size, 1, 1, seq_len].
    # Distinguishes between non-padding tokens (with a value of 0) and padding tokens
    # (with a value of a large negative number).
    extended_attention_mask: torch.Tensor = get_extended_attention_mask(attention_mask, self.dtype)

    # Pass the hidden states through the encoder layers.
    for i, layer_module in enumerate(self.gpt_layers):
      # Feed the encoding from the last bert_layer to the next.
      hidden_states = layer_module(hidden_states, extended_attention_mask)

    return hidden_states

  def forward(self, input_ids, attention_mask):
    """
    input_ids: [batch_size, seq_len], seq_len is the max length of the batch
    attention_mask: same size as input_ids, 1 represents non-padding tokens, 0 represents padding tokens
    """
    # Get the embedding for each input token.
    embedding_output = self.embed(input_ids=input_ids)

    # Feed to a transformer (a stack of GPTLayers).
    sequence_output = self.encode(embedding_output, attention_mask=attention_mask)
    sequence_output = self.final_layer_norm(sequence_output)

    # Get the hidden state of the final token.
    last_non_pad_idx = attention_mask.sum(dim=1) - 1  # Subtract 1 to get last index
    last_token = sequence_output[torch.arange(sequence_output.shape[0]), last_non_pad_idx]

    return {'last_hidden_state': sequence_output, 'last_token': last_token}

  def hidden_state_to_token(self, hidden_state):
    """
    TODO-5: Convert hidden states back to token logits.

    Implementation hints: 
    - GPT-2 uses weight tying with the input word embeddings. 
    - The logits are the dot product between output hidden states and the word embedding weights: hidden_state(s) * E^T
    """
    ### YOUR CODE HERE
    logits = torch.matmul(hidden_state, self.word_embedding.weight.T)
    return logits

  @classmethod
  def from_pretrained(cls, model='gpt2', d=768, l=12, num_heads=12):
    gpt_model = OpenAIGPT2Model.from_pretrained(model).eval()
    our_model = GPT2Model(GPT2Config(hidden_size=d, num_hidden_layers=l,num_attention_heads=num_heads,
                                     intermediate_size=d*3)).eval()

    # Load word and positional embeddings.
    our_model.word_embedding.load_state_dict(gpt_model.wte.state_dict())
    our_model.pos_embedding.load_state_dict(gpt_model.wpe.state_dict())

    for i in range(l):
      l = our_model.gpt_layers[i]
      # Remap the Q,K,V weights from a conv1d to 3 linear projections
      l.self_attention.query.weight.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.weight'][:, :d].T
      l.self_attention.query.bias.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.bias'][:d]
      l.self_attention.key.weight.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.weight'][:, d:d*2].T
      l.self_attention.key.bias.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.bias'][d:d*2]
      l.self_attention.value.weight.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.weight'][:, d*2:].T
      l.self_attention.value.bias.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.bias'][d*2:]

      # Remap final dense layer in MHA.
      l.attention_dense.weight.data = gpt_model.state_dict()[f'h.{i}.attn.c_proj.weight'].T
      l.attention_dense.bias.data = gpt_model.state_dict()[f'h.{i}.attn.c_proj.bias']

      # Remap attention layer norm.
      l.attention_layer_norm.weight.data = gpt_model.state_dict()[f'h.{i}.ln_1.weight']
      l.attention_layer_norm.bias.data = gpt_model.state_dict()[f'h.{i}.ln_1.bias']

      # Remap post-attention MLP layers.
      l.interm_dense.weight.data = gpt_model.state_dict()[f'h.{i}.mlp.c_fc.weight'].T
      l.interm_dense.bias.data = gpt_model.state_dict()[f'h.{i}.mlp.c_fc.bias']
      l.out_dense.weight.data = gpt_model.state_dict()[f'h.{i}.mlp.c_proj.weight'].T
      l.out_dense.bias.data = gpt_model.state_dict()[f'h.{i}.mlp.c_proj.bias']

      # Remap second layer norm weights.
      l.out_layer_norm.weight.data = gpt_model.state_dict()[f'h.{i}.ln_2.weight']
      l.out_layer_norm.bias.data = gpt_model.state_dict()[f'h.{i}.ln_2.bias']

    # Remap the final layer norm values.
    our_model.final_layer_norm.weight.data = gpt_model.state_dict()['ln_f.weight']
    our_model.final_layer_norm.bias.data = gpt_model.state_dict()['ln_f.bias']

    return our_model

In [8]:
# Sanity check: compare with Huggingface GPT2 implementation
def test_gpt2(model_size='gpt2'):
  sent_ids = torch.tensor([[101, 7592, 2088, 102, 0, 0, 0, 0],
                           [101, 7592, 15756, 2897, 2005, 17953, 2361, 102]])
  att_mask = torch.tensor([[1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1]])

  # Load both the OpenAI and your own model.
  openai_model = OpenAIGPT2Model.from_pretrained(model_size)
  gpt = GPT2Model.from_pretrained(model=model_size, **model_size_to_params(model_size))

  outputs = gpt(sent_ids, att_mask)
  openai_outputs = openai_model(input_ids=sent_ids, attention_mask=att_mask, output_hidden_states=True).hidden_states[-1]

  att_mask = att_mask.unsqueeze(-1)
  outputs['last_hidden_state'] = outputs['last_hidden_state'] * att_mask
  openai_outputs *= att_mask

  assert torch.allclose(outputs['last_hidden_state'], openai_outputs, atol=1e-1, rtol=1e-2)

  print("Your GPT2 implementation is correct!")

test_gpt2('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Your GPT2 implementation is correct!


## Adam Optimizer

In [31]:
class AdamW(Optimizer):
    def __init__(
            self,
            params: Iterable[torch.nn.parameter.Parameter],
            lr: float = 1e-3,
            betas: Tuple[float, float] = (0.9, 0.999),
            eps: float = 1e-6,
            weight_decay: float = 0.0,
            correct_bias: bool = True,
    ):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {} - should be >= 0.0".format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[1]))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {} - should be >= 0.0".format(eps))
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, correct_bias=correct_bias)
        super().__init__(params, defaults)

    def step(self, closure: Callable = None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")

                # State should be stored in this dictionary.
                state = self.state[p]

                # Access hyperparameters from the `group` dictionary.
                lr = group["lr"]
                eps = group["eps"]
                weight_decay = group["weight_decay"]
                correct_bias = group["correct_bias"]
                beta1, beta2 = group["betas"]
                state = self.state[p]
                if len(state) == 0:
                    state["step"] = 0
                    state["exp_avg"] = torch.zeros_like(p.data)
                    state["exp_avg_sq"] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state["exp_avg"], state["exp_avg_sq"]
                
                state["step"] += 1
                t = state["step"]
                
                """
                TODO-6: Implement the AdamW parameter update for this step.

                Implementation hints:
                1. Update biased first moment estimate:
                    m_t = beta1 * m_{t-1} + (1 - beta1) * grad
                2. Update biased second raw moment estimate:
                    v_t = beta2 * v_{t-1} + (1 - beta2) * grad^2
                3. Apply bias correction if correct_bias=True:
                    m_hat = m_t / (1 - beta1^t)
                    v_hat = v_t / (1 - beta2^t)
                4. Compute step size:
                    step_size = lr (or lr / (1 - beta1^t) if bias correction)
                5. Update parameters:
                    p = p - step_size * m_hat / (sqrt(v_hat) + eps)
                6. Apply decoupled weight decay after the parameter update (if weight_decay > 0):
                    p = p - lr * weight_decay * p
                Reference:
                Algorithm 1 in "Adam: A Method for Stochastic Optimization"
                https://arxiv.org/abs/1412.6980
                """
                ### YOUR CODE HERE
                exp_avg = state["exp_avg"]
                exp_avg_sq = state["exp_avg_sq"]

                # update biased first and second moments
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                # bias correction
                if correct_bias:
                    m_hat = exp_avg / (1 - beta1 ** t)
                    v_hat = exp_avg_sq / (1 - beta2 ** t)
                else:
                    m_hat = exp_avg
                    v_hat = exp_avg_sq

                # update p
                p.data.addcdiv_(-lr, m_hat, torch.sqrt(v_hat) + eps)

                # apply weight decay
                if weight_decay > 0:
                    p.data.add_(-lr * weight_decay, p.data)
        return loss

In [10]:
# Sanity check for AdamW optimizer
def test_optimizer(opt_class) -> torch.Tensor:
    rng = np.random.default_rng(SEED)
    torch.manual_seed(SEED)
    model = torch.nn.Linear(3, 2, bias=False)
    opt = opt_class(
        model.parameters(),
        lr=1e-3,
        weight_decay=1e-4,
        correct_bias=True,
    )
    for i in range(1000):
        opt.zero_grad()
        x = torch.FloatTensor(rng.uniform(size=[model.in_features]))
        y_hat = model(x)
        y = torch.Tensor([x[0] + x[1], -x[2]])
        loss = ((y - y_hat) ** 2).sum()
        loss.backward()
        opt.step()
    return model.weight.detach()

SEED = 0
ref = torch.tensor(np.load("optimizer_test.npy"))
actual = test_optimizer(AdamW)
print(ref)
print(actual)
assert torch.allclose(ref, actual, atol=1e-6, rtol=1e-4)
print("Optimizer test passed!")

/tmp/ipykernel_40/3373583738.py:93: UserWarning: This overload of addcdiv_ is deprecated:
	addcdiv_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcdiv_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1519.)
  p.data.addcdiv_(-lr, m_hat, torch.sqrt(v_hat) + eps)


tensor([[ 0.5548,  0.8667,  0.0729],
        [-0.4472, -0.2951, -0.2717]])
tensor([[ 0.5548,  0.8667,  0.0729],
        [-0.4472, -0.2951, -0.2717]])
Optimizer test passed!


## Model Loading & Text Generation

In [6]:
@torch.no_grad()
def generate_gpt2(model, tokenizer, input_ids, max_gen_length=50, device="cuda"):
    """
    Generate text from a GPT-2 model given a single input sequence (greedy decoding).

    Note:
        - Currently only supports batch_size=1 (single input sequence).
        - Using greedy decoding, so each run with the same input produces the same output.
        - Other sampling-based decoding methods (e.g., top-k, top-p, temperature) can introduce randomness and yield different outputs each run.

    Args:
        model: GPT-2 model (pretrained or fine-tuned)
        tokenizer: GPT-2 tokenizer
        input_ids: torch.LongTensor of shape [1, seq_len], input token IDs
        max_gen_length: int, maximum number of tokens to generate
        device: str, "cuda" or "cpu"

    Returns:
        str: Generated text
    """
    model.eval()
    input_ids = input_ids.to(device)  # move input to device
    output_ids = input_ids.clone()

    """
    TODO-8: Greedy next-token generation loop

    Implementation hints:
    Repeat the below steps up to max_gen_length:
    1. Construct an attention mask based on current output_ids (non-pad tokens).
    2. Pass output_ids and attention_mask through the model to get hidden states.
    3. Convert the last hidden state to logits over the vocabulary using model.hidden_state_to_token.
    4. Select the next token using greedy decoding (argmax over logits).
    5. Append the next token to output_ids.
    6. Stop the loop early if the EOS token is generated.
    
    """
    ### YOUR CODE HERE
    for _ in range(max_gen_length):
        attention_mask=torch.ones_like(output_ids)
        hidden_states=model(output_ids,attention_mask) #hidden state is a dictionary w keys last_hidden_state and last_token
        # print(type(hidden_states))
        # print(hidden_states.keys() if hasattr(hidden_states, 'keys') else hidden_states)
        last_hidden = hidden_states['last_hidden_state'][:, -1, :] #shape : [batch_size, sequence_length, hidden_dim]
        logits = model.hidden_state_to_token(last_hidden)
        next_token = torch.argmax(logits,dim=-1).unsqueeze(-1)
        output_ids = torch.cat([output_ids, next_token],dim=-1)
        if (next_token == tokenizer.eos_token_id).any():
            break

    # raise NotImplementedError

    # Decode generated tokens to string
    ids = output_ids[0]
    text = tokenizer.decode(ids, skip_special_tokens=True)

    return text

In [12]:
# # Load a pretrained GPT-2 model with official weights
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token
# model = GPT2Model.from_pretrained("gpt2").to(DEVICE)

## Load NLI Dataset

In [7]:
def compute_accuracy(preds, labels):
    correct = sum(p.lower().strip() == l.lower().strip() for p, l in zip(preds, labels))
    return correct / len(labels)

def evaluate_gpt2_xnli(model, tokenizer, dataloader, max_gen_length=10, device="cuda"):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for item in tqdm(dataloader, desc="Generating"):
            input_ids = item['input_ids']
            gen_text = generate_gpt2(model, tokenizer, input_ids, max_gen_length=max_gen_length, device=device)
            pred_label = gen_text.split("Label:")[-1].strip()
            all_preds.append(pred_label)
            all_labels.extend(item['label_strs'])
    acc = compute_accuracy(all_preds, all_labels)
    print(f"Evaluation accuracy: {acc*100:.2f}%")
    return acc, all_preds, all_labels

class XNLIDataset(Dataset):
    """
    A PyTorch Dataset for XNLI (Cross-lingual Natural Language Inference) task.

    Supports train, dev, and test splits in a specific language, 
    tokenizes text inputs for GPT-style models, and optionally subsamples the dataset.

    Attributes:
        split (str): Dataset split, one of 'train', 'dev', 'test'.
        lang (str): Language code (e.g., 'en', 'zh').
        tokenizer: A HuggingFace tokenizer to convert text to input IDs.
        max_length (int): Maximum sequence length for tokenization.
        LABEL2ID (dict): Mapping from textual labels to integer IDs.
        ID2LABEL (dict): Reverse mapping from integer IDs to textual labels.
        data (pd.DataFrame): The loaded and preprocessed dataset.
    """
    def __init__(
        self,
        split="train",
        lang="en",
        train_path_template="XNLI-MT-1.0/multinli/multinli.train.{lang}.tsv",
        test_path="XNLI-1.0/xnli.test.tsv",
        dev_path="XNLI-1.0/xnli.dev.tsv",
        tokenizer=None,
        max_length=1024,
        subset = 1.0  # 0~1
    ):
        self.split = split
        self.lang = lang
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.LABEL2ID = {"entailment": 0, "contradictory": 1, "neutral": 2}
        self.ID2LABEL = {v: k for k, v in self.LABEL2ID.items()}

        if split == "train":
            path = train_path_template.format(lang=lang)
            df = self.read_xnli_tsv(path, split)
            df = df.dropna(subset=['premise','hypo','label'])
        elif split in ["dev", "test"]:
            path = test_path if split=="test" else dev_path
            df = self.read_xnli_tsv(path, split)
            df = df[df['language']==lang].copy()
            keep_cols = ['sentence1', 'sentence2', 'gold_label']
            df = df[keep_cols].dropna()
            df.rename(columns={'sentence1':'premise','sentence2':'hypo','gold_label':'label'}, inplace=True)
            df['label'] = df['label'].replace({'contradiction': 'contradictory'})
        else:
            raise ValueError("split must be one of ['train','dev','test']")
        
        original_num = len(df)
        if subset < 1.0:
            n = max(1, int(len(df) * subset))
            df = df.iloc[:n].reset_index(drop=True)
        subset_num = len(df)

        self.data = df.reset_index(drop=True)
        print(f"Dataset initialized: split='{split}', lang='{lang}', total={original_num}, subset={subset}, subset_count={subset_num}")

    def read_xnli_tsv(self, path, split):
        """
        Read an XNLI TSV file and return it as a pandas DataFrame.

        Args:
            path (str): Path to the TSV file.
            split (str): One of "train", "dev", "test" indicating the dataset split.

        Returns:
            pd.DataFrame: The dataset as a DataFrame with appropriate columns.
        """
        if split == "train":
            with open(path, "r", encoding="utf-8") as f:
                lines = f.read().splitlines()
            header = lines[0].split("\t")
            data = []
            for i, line in enumerate(lines[1:], start=2):
                parts = line.split("\t")
                if len(parts) == len(header):
                    data.append(parts)
                else:
                    print(f"skip row {i}: {len(parts)} cols → {parts[:2]}")
        else:
            with open(path, "r", encoding="utf-8") as f:
                reader = csv.reader(f, delimiter="\t")
                rows = list(reader)
            header = rows[0]
            expected_cols = len(header)
            data = []
            for i, row in enumerate(rows[1:], start=2):
                if len(row) == expected_cols:
                    data.append(row)
                else:
                    print(f"skip row {i}: {len(row)} cols → {row[:2]}")
        return pd.DataFrame(data, columns=header)

    def __len__(self):
        """Return the number of examples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieve a single example by index and tokenize it.

        For training split:
            - Constructs the input as "Premise: ... Hypothesis: ... Label: ..."
            - Tokenizes the full input.
            - Masks the prefix tokens in the labels with -100 for GPT loss computation.

        For dev/test split:
            - Constructs the input without label as "Premise: ... Hypothesis: ... Label:"

        Returns:
            dict: Contains 'input_ids', 'attention_mask', 'labels' (train only), 'label_str'
        """
        row = self.data.iloc[idx]
        premise = row['premise']
        hypo = row['hypo']
        label = row['label']
        if self.lang == 'zh': # de-tokenize for Chinese
            premise = premise.replace(" ", "")
            hypo = hypo.replace(" ", "")

        if self.split == "train":
            prefix = f"Premise: {premise}\nHypothesis: {hypo}\nLabel:"
            full_text = prefix + str(self.LABEL2ID[label])
            tokenized = self.tokenizer(
                full_text,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt"
            )
            tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}

            prefix_ids = self.tokenizer(prefix).input_ids
            labels_ids = tokenized['input_ids'].clone()
            labels_ids[:len(prefix_ids)] = -100 # Masks the prefix tokens in the labels with -100 for GPT loss computation.
            tokenized['labels'] = labels_ids
            tokenized['label_str'] = str(self.LABEL2ID[label])
            return tokenized
        else:
            text = f"Premise: {premise}\nHypothesis: {hypo}\nLabel:"
            tokenized = self.tokenizer(
                text,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt"
            )
            tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}
            tokenized['label_str'] = str(self.LABEL2ID[label])
            return tokenized

    @staticmethod
    def collate_fn(batch):
        """
        Collate a batch of examples into padded tensors.

        Pads 'input_ids' and 'attention_mask' to the max length in the batch.
        Pads 'labels' with -100 if present.
        Collects 'label_str' for reference.

        Returns:
            dict: Padded tensors and label strings for the batch.
        """
        input_ids = torch.nn.utils.rnn.pad_sequence(
            [b['input_ids'] for b in batch],
            batch_first=True,
            padding_value=0
        )
        attention_mask = torch.nn.utils.rnn.pad_sequence(
            [b['attention_mask'] for b in batch],
            batch_first=True,
            padding_value=0
        )

        if 'labels' in batch[0]:
            labels = torch.nn.utils.rnn.pad_sequence(
                [b['labels'] for b in batch],
                batch_first=True,
                padding_value=-100
            )
        else:
            labels = None

        label_strs = [b['label_str'] for b in batch]

        out = {"input_ids": input_ids, "attention_mask": attention_mask, "label_strs": label_strs}
        if labels is not None:
            out["labels"] = labels
        return out

## Zero-shot Cross-lingual Transfer

In [10]:
langs = ['en', 'ar', 'bg', 'de','el','es','fr','hi','ru','sw','th','tr','ur','vi','zh']

In [14]:
TEST_SUBSET = 1
path = "./best_model/model_en_epoch_3.pt" # To-do: Swap this naming convension

finetuned_model = GPT2Model(GPT2Config()).to(DEVICE)
finetuned_model.load_state_dict(torch.load(path))

all_test_datasets = {}
all_test_loader = {}
for lang in langs:
    test_dataset = XNLIDataset(split="test", lang=lang, tokenizer=tokenizer, max_length=1024, subset=TEST_SUBSET)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=XNLIDataset.collate_fn)
    all_test_datasets[lang] = test_dataset
    all_test_loader[lang] = test_loader

all_results = {}
for lang in langs:
    test_loader = all_test_loader[lang]
    if lang == "en":
        print(f"Evaluating on {lang}...")
    else:
        print(f"Evaluating zero-shot on {lang}...")
    acc, all_preds, all_labels = evaluate_gpt2_xnli(finetuned_model, tokenizer, test_loader, max_gen_length=1, device=DEVICE)
    all_results[lang] = acc

print("Zero-shot cross-lingual accuracy per language:")
for lang, acc in all_results.items():
    print(f"{lang}: {acc*100:.2f}%")

Dataset initialized: split='test', lang='en', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='ar', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='bg', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='de', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='el', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='es', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='fr', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='hi', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='ru', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='sw', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='th', total=5010, subset=1, subset_count=5010
Dataset initialized: split='test', lang='tr', total=50

Generating: 100%|██████████| 5010/5010 [00:47<00:00, 106.09it/s]


Evaluation accuracy: 83.23%
Evaluating zero-shot on ar...


Generating: 100%|██████████| 5010/5010 [00:53<00:00, 92.80it/s] 


Evaluation accuracy: 37.19%
Evaluating zero-shot on bg...


...

Evaluation accuracy: 34.93%
Evaluating zero-shot on vi...


Generating: 100%|██████████| 5010/5010 [00:53<00:00, 93.16it/s] 


Evaluation accuracy: 39.22%
Evaluating zero-shot on zh...


Generating: 100%|██████████| 5010/5010 [00:51<00:00, 96.43it/s] 

Evaluation accuracy: 37.94%
Zero-shot cross-lingual accuracy per language:
en: 83.23%
ar: 37.19%
bg: 37.72%
de: 40.28%
el: 35.79%
es: 46.47%
fr: 43.83%
hi: 34.55%
ru: 37.52%
sw: 39.92%
th: 34.85%
tr: 40.50%
ur: 34.93%
vi: 39.22%
zh: 37.94%


### Extra: Comparing fine-tuned model from Task 2 with Chat-GPT's default weights

In [54]:
@torch.no_grad()
def generate_label(model, tokenizer, input_ids, attention_mask, max_gen_length=1):
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_gen_length,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False  # Greedy decoding
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
                            
                            

def evaluate_official_gpt2_xnli(model, tokenizer, dataloader, max_gen_length=1):
    model.eval()
    all_preds, all_labels = [], []

    for batch in tqdm(dataloader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        label_strs = batch["label_strs"]

        gen_text = generate_label(model, tokenizer, input_ids, attention_mask, max_gen_length)
        pred_label = gen_text.split("Label:")[-1].strip()  # Extract after "Label:"
        all_preds.append(pred_label)
        all_labels.extend(label_strs)

    # Compute accuracy
    correct = sum(p == l for p, l in zip(all_preds, all_labels))
    acc = correct / len(all_labels)
    return acc, all_preds, all_labels


In [55]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token  # Set pad token to EOS

official_model = GPT2LMHeadModel.from_pretrained("gpt2").to(DEVICE).eval()


all_results = {}
for lang in langs:
    print(f"Evaluating zero-shot on {lang}...")
    acc, preds, labels = evaluate_official_gpt2_xnli(model, tokenizer, all_test_loader[lang])
    all_results[lang] = acc

print("\nZero-shot cross-lingual accuracy:")
for lang, acc in all_results.items():
    print(f"{lang}: {acc*100:.2f}%")


Evaluating zero-shot on en...


Evaluating: 100%|██████████| 5010/5010 [01:31<00:00, 54.80it/s]


Evaluating zero-shot on ar...


Evaluating: 100%|██████████| 5010/5010 [01:41<00:00, 49.27it/s]


Evaluating zero-shot on bg...


...

Evaluating zero-shot on vi...


Evaluating: 100%|██████████| 5010/5010 [01:42<00:00, 48.65it/s]


Evaluating zero-shot on zh...


Evaluating: 100%|██████████| 5010/5010 [01:35<00:00, 52.41it/s]


Zero-shot cross-lingual accuracy:
en: 0.02%
ar: 0.00%
bg: 0.00%
de: 0.00%
el: 0.00%
es: 0.02%
fr: 0.02%
hi: 0.00%
ru: 0.00%
sw: 0.00%
th: 0.00%
tr: 0.02%
ur: 0.02%
vi: 0.04%
zh: 0.00%


## Fertility-based Language Selection

Guidance: You may notice that some languages achieve reasonable zero-shot cross-lingual performance. This is likely because these languages are closer to English (e.g., in writing system), making cross-lingual transfer from English easier. However, many other languages perform close to random guessing, which is expected since GPT-2 was pretrained entirely on English data.

To perform further multilingual fine-tuning, we need to identify which languages GPT-2 can realistically support (because if a language is not supported, fine-tuning on it will have little effect). A straightforward way to check this is to inspect the tokens in the model’s tokenizer. However, this is not practical for GPT-2-like models, because they use a Byte-Pair Encoding (BPE) tokenizer. BPE can decompose any Unicode string into subwords, even if the string never appeared in training, making it difficult to determine whether a language is truly supported.

Instead, we can approximate tokenizer support using fertility, a metric that measures the average number of subwords produced per word. Lower fertility indicates better tokenizer quality and compression, while high fertility suggests heavy fragmentation, which can hurt model performance. By combining fertility analysis with zero-shot cross-lingual results, we can identify a subset of languages that GPT-2 can reasonably handle (a rough estimate, as officially GPT-2 is designed for English). Then, we can proceed with multilingual fine-tuning experiments on these languages.

Reference: How Good is Your Tokenizer? On the Monolingual Performance of Multilingual Language Models

In [15]:
def compute_fertility(dataset, tokenizer):
    """
    Compute average fertility for a dataset.
    Fertility = #tokens / #words
    Note: word splitting is approximate and uses whitespace.
    """
    total_words = 0
    total_tokens = 0
    samples = len(dataset)
    
    for i in tqdm(range(samples), desc="Computing fertility"):
        row = dataset.data.iloc[i]
        for sent in [row['premise'], row['hypo']]:
            words = sent.strip().split()  # crude word estimate
            tokens = tokenizer.tokenize(sent)
            total_words += len(words)
            total_tokens += len(tokens)
    
    fertility = total_tokens / total_words if total_words > 0 else 0.0
    return fertility

In [29]:
subset_for_check = 0.01
path = "./best_model/model.pt"

finetuned_model = GPT2Model(GPT2Config()).to(DEVICE)
finetuned_model.load_state_dict(torch.load(path))

all_results = {}
for lang in langs:
    train_dataset = XNLIDataset(
        split="train",
        lang=lang,
        tokenizer=tokenizer,
        subset=subset_for_check
    )
    fertility_score = compute_fertility(train_dataset, tokenizer)
    print(f"{lang}: fertility = {fertility_score:.2f}")
    all_results[lang] = fertility_score
    
print("\nFertility scores:")
for lang, fertility_score in all_results.items():
    print(f"{lang}: {fertility_score:.2f}")

Dataset initialized: split='train', lang='en', total=392702, subset=0.01, subset_count=3927


Computing fertility: 100%|██████████| 3927/3927 [00:00<00:00, 4278.04it/s]


en: fertility = 1.11
Dataset initialized: split='train', lang='ar', total=392702, subset=0.01, subset_count=3927


Computing fertility: 100%|██████████| 3927/3927 [00:00<00:00, 3947.75it/s]


ar: fertility = 4.70
Dataset initialized: split='train', lang='bg', total=392702, subset=0.01, subset_count=3927


...

ur: fertility = 5.16
Dataset initialized: split='train', lang='vi', total=392702, subset=0.01, subset_count=3927


Computing fertility: 100%|██████████| 3927/3927 [00:01<00:00, 3551.02it/s]


vi: fertility = 3.62
Dataset initialized: split='train', lang='zh', total=392702, subset=0.01, subset_count=3927


Computing fertility: 100%|██████████| 3927/3927 [00:00<00:00, 3954.57it/s]

zh: fertility = 3.77

Fertility scores:
en: 1.11
ar: 4.70
bg: 5.53
de: 2.10
el: 6.16
es: 1.83
fr: 1.75
hi: 5.12
ru: 5.90
sw: 2.08
th: 9.48
tr: 2.73
ur: 5.16
vi: 3.62
zh: 3.77


## Fine-tune GPT-2 (per-language)

Guidance: Load the pretrained GPT-2 (not the ones fine-tuned on English NLI) along with the training data for a single target language. Choose non-English languages that performed well in the zero-shot cross-lingual transfer and fertility evaluation. It depends on you how many languages to include. Fine-tune a separate model for each selected language. Afterwards, compare these per-language fine-tuned models with the zero-shot cross-lingual transfer results.

### Generic retraining pipeline
1. Load dataset from `XNLIDataset` and split it into test and train.
2. Use DataLoader to load it up.
3. Use Adam optimiser from the function `AdamW(Optimizer)`.
4. Build the training loop (intitialise weights, train, get loss, gradietn descent, early stopping based on dev_dataset accuracy/results).
5. Save the best checkpoint per langauge
6. Evaluate via test_set

Then comapre with its zero-shot counterpart.


In [8]:
from transformers import get_linear_schedule_with_warmup

# Adjust from 0 to 1, but consider 0.1 for rough testing of pipeline
TRAIN_SUBSET = 1 #0.01 for testing, 1 for actual fine-tuning
DEV_SUBSET = 1
TEST_SUBSET = 1

# Hyperparamter of gpt2 fine-tuning
EPOCHS = 5 # 2 for testing, 1 for actual fine-tuning
BATCH_SIZE = 8
LR = 2e-5
WEIGHT_DECAY = 0.01
WARMUP_RATIO=0.1
CORRECT_BIAS = True
SAVE_DIR = "best_model"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
VOCAB_SIZE = tokenizer.vocab_size


def split_dataset(lang, tokenizer):
    train_dataset = XNLIDataset(
        split="train",
        lang=lang,
        tokenizer=tokenizer,
        subset=TRAIN_SUBSET
    )

    dev_dataset = XNLIDataset(
        split="dev",
        lang=lang,
        tokenizer=tokenizer,
        subset=DEV_SUBSET
    )

    test_dataset = XNLIDataset(
        split="test",
        lang=lang,
        tokenizer=tokenizer,
        subset=TEST_SUBSET
    )
    
    return train_dataset, dev_dataset, test_dataset


def full_finetuning_pipeline(train_dataset, dev_dataset, lang):
    # Create DataLoaders for training and validation datasets
    train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,collate_fn=XNLIDataset.collate_fn)
    dev_loader = DataLoader(dev_dataset,shuffle=False,collate_fn=XNLIDataset.collate_fn)

    # Initialize optimizer
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, correct_bias=CORRECT_BIAS)
    # Include early stopping
    patience = 2
    epochs_without_improvement = 0
    # Track training progress
    global_train_losses = []
    total_train_loss = 0.0
    total_train_steps = 0
    print_interval = 10

    # Track best dev accuracy for model saving
    # This only works for epoch > 1 
    best_dev_acc = 0.0
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    # Scheduler
    total_steps = len(train_loader) * EPOCHS
    warmup_steps = int(total_steps * WARMUP_RATIO)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, warmup_steps, total_steps
    )

    # Training loop
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch+1}/{EPOCHS}")
        model.train()
        # Iterate over batches
        loop = tqdm(train_loader, desc="Training")
        for batch in loop:
            input_ids = batch["input_ids"].to(DEVICE)        # [B, seq_len]
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch.get("labels").to(DEVICE)                    # [B, seq_len]

            optimizer.zero_grad()

            hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']  # [B, seq_len, hidden]

            """
            Reused from to-do 9.

            Implementation hints:
            1. Convert hidden states to logits over the vocabulary using model.hidden_state_to_token.
            2. Shift logits and labels for next-token prediction to align each prediction with the correct next token.
            3. Compute the cross-entropy loss, making sure positions with label=-100 are ignored.
            4. Backpropagate and update model parameters.
            """

            ### YOUR CODE HERE
            logits = model.hidden_state_to_token(hidden_states)
            shifted_logits = logits[:, :-1, :] #take all but last token cause mismatch right now -> output pred 0, pred 1, pred 2
            shifted_labels = labels[:,1:] #take all but first token                                labels word1 , word 2, word 3
            loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)                                  #but we want pred 0 to predict word 1, pred 1 to predict word 2, pred 2 to predict word 3, so remove first label so labels look like [word2,word3..] and predictions remove last since there is no more word after last word.
            loss = loss_fn(shifted_logits.reshape(-1,logits.size(-1)), shifted_labels.reshape(-1))  # flatten to [B*(seq_len-1), vocab_size] since N= B*(seq_len-1) for logits and [B*(seq_len-1)] for labels
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            # raise NotImplementedError     
            total_train_loss += loss.item()
            total_train_steps += 1
            global_train_avg_loss = total_train_loss / total_train_steps
            global_train_losses.append(global_train_avg_loss)

            loop.set_postfix({'avg_loss': f"{global_train_avg_loss:.4f}"})

        print(f"Epoch {epoch+1} finished | Global Avg Loss: {global_train_avg_loss:.4f}")

        acc, all_preds, all_labels = evaluate_gpt2_xnli(model, tokenizer, dev_loader, max_gen_length=1, device=DEVICE)

        
        # Save model at every epoch
        torch.save(model.state_dict(), f"{SAVE_DIR}/model_{lang}_epoch_{epoch}.pt")
        # Save best model
        if acc > best_dev_acc:
            best_dev_acc = acc
        #     torch.save(model.state_dict(), f"{SAVE_DIR}/model_{lang}.pt")
        #     print(f"New best model saved at {SAVE_DIR}/model_{lang}.pt with dev accuracy {best_dev_acc*100:.2f}%")
        # Or trigger early stopping
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print("Early stopping triggered.")
        # break
        
        
def compute_fertility(dataset, tokenizer):
    total_words = 0
    total_tokens = 0
    samples = len(dataset)
    
    for i in tqdm(range(samples), desc="Computing fertility"):
        row = dataset.data.iloc[i]
        for sent in [row['premise'], row['hypo']]:
            words = sent.strip().split()  # crude word estimate
            tokens = tokenizer.tokenize(sent)
            total_words += len(words)
            total_tokens += len(tokens)
    
    fertility = total_tokens / total_words if total_words > 0 else 0.0
    return fertility


In [ ]:
# Main loop
import time

# Empty cache before starting
torch.cuda.empty_cache()

# Languages not supported: hindi, urdu, bulgurian, russian, greek, thai
# Original list: ['ar','de','es','fr','sw','tr','vi','zh']
chosen_lang = ['vi', 'zh', 'es'] # "vi", "zh", "es"
all_results = {}

for lang in chosen_lang:
    # Load a pretrained GPT-2 model with official weights
    model = GPT2Model.from_pretrained("gpt2").to(DEVICE)
    
    # Split train, val and test
    train_dataset, dev_dataset, test_dataset = split_dataset(lang, tokenizer)
    
    # Full finetunign pipeline, refer to code above for params
    start_time = time.time()
    full_finetuning_pipeline(train_dataset, dev_dataset, lang)
    end_time = time.time()
    
    # Perform testing
    path = f"./{SAVE_DIR}/model_{lang}.pt"
    model.load_state_dict(torch.load(path))
    model.eval
    test_loader = DataLoader(test_dataset,shuffle=False,collate_fn=XNLIDataset.collate_fn)
    acc, all_preds, all_labels = evaluate_gpt2_xnli(model, tokenizer, test_loader, max_gen_length=1, device=DEVICE)
    fertility_score = compute_fertility(train_dataset, tokenizer)
    time_taken = end_time - start_time
    all_results[lang] = [acc, time_taken, fertility_score]
    
    print(f"\nZero-shot cross-lingual accuracy {lang}: {acc*100:.2f}%")
    print(f"Training took {(time_taken)/60:.2f} minutes for {EPOCHS} epochs")
    print(f"Fertility score: {fertility_score:.2f}")
    
    # break
    

for lang, data in all_results.items():
    # First data denotes accuracy
    print(f"\nZero-shot cross-lingual accuracy {lang}: {data[0]*100:.2f}%")
    # Second denotes time
    print(f"Training took {(data[1])/60:.2f} minutes for {EPOCHS} epochs")
    # Third denotes fertility score
    print(f"Fertility score: {data[2]:.2f}")
    
    

Dataset initialized: split='train', lang='vi', total=392702, subset=1, subset_count=392702
Dataset initialized: split='dev', lang='vi', total=2490, subset=1, subset_count=2490
Dataset initialized: split='test', lang='vi', total=5010, subset=1, subset_count=5010
Epoch 1/5


Training: 100%|██████████| 49088/49088 [2:23:05<00:00,  5.72it/s, avg_loss=0.9659]  


Epoch 1 finished | Global Avg Loss: 0.9659


Generating: 100%|██████████| 2490/2490 [00:23<00:00, 106.72it/s]


Evaluation accuracy: 63.05%
Epoch 2/5


...

Training: 100%|██████████| 49088/49088 [2:23:03<00:00,  5.72it/s, avg_loss=0.8051]  


Epoch 4 finished | Global Avg Loss: 0.8051


Generating: 100%|██████████| 2490/2490 [00:22<00:00, 109.73it/s]


Evaluation accuracy: 66.39%
Epoch 5/5


Training:  74%|███████▎  | 36087/49088 [1:45:00<37:08,  5.83it/s, avg_loss=0.7848]  

## Code-switched test with custom CSDataset & testing

In [9]:
class CSDataset(Dataset):
    def __init__(
        self,
        split="test",
        lang="zh",
        test_path="cs_dataset/cs_{lang}_test.tsv",
        tokenizer=None,
        max_length=1024,
        subset = 1.0  # 0~1
    ):
        self.split = split
        self.lang = lang
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.LABEL2ID = {"entailment": 0, "contradictory": 1, "neutral": 2}
        self.ID2LABEL = {v: k for k, v in self.LABEL2ID.items()}

        if split=="test":
            path = test_path.format(lang=self.lang)
            df = self.read_dataset_tsv(path, split)
            keep_cols = ['sentence1', 'sentence2', 'gold_label']
            df = df[keep_cols].dropna()
            df.rename(columns={'sentence1':'premise','sentence2':'hypo','gold_label':'label'}, inplace=True)
            df['label'] = df['label'].replace({'contradiction': 'contradictory'})
        else:
            raise ValueError("split must be one of ['train','dev','test']")
        
        original_num = len(df)
        if subset < 1.0:
            n = max(1, int(len(df) * subset))
            df = df.iloc[:n].reset_index(drop=True)
        subset_num = len(df)

        self.data = df.reset_index(drop=True)
        print(f"Dataset initialized: split='{split}', lang='{lang}', total={original_num}, subset={subset}, subset_count={subset_num}")

    def read_dataset_tsv(self, path, split):
        if split == "train":
            with open(path, "r", encoding="utf-8") as f:
                lines = f.read().splitlines()
            header = lines[0].split("\t")
            data = []
            for i, line in enumerate(lines[1:], start=2):
                parts = line.split("\t")
                if len(parts) == len(header):
                    data.append(parts)
                else:
                    print(f"skip row {i}: {len(parts)} cols → {parts[:2]}")
        else:
            with open(path, "r", encoding="utf-8") as f:
                reader = csv.reader(f, delimiter="\t")
                rows = list(reader)
            header = rows[0]
            expected_cols = len(header)
            data = []
            for i, row in enumerate(rows[1:], start=2):
                if len(row) == expected_cols:
                    data.append(row)
                else:
                    print(f"skip row {i}: {len(row)} cols → {row[:2]}")
        return pd.DataFrame(data, columns=header)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        premise = row['premise']
        hypo = row['hypo']
        label = row['label']
        if self.lang == 'zh': # de-tokenize for Chinese
            premise = premise.replace(" ", "")
            hypo = hypo.replace(" ", "")

        if self.split == "train":
            prefix = f"Premise: {premise}\nHypothesis: {hypo}\nLabel:"
            full_text = prefix + str(self.LABEL2ID[label])
            tokenized = self.tokenizer(
                full_text,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt"
            )
            tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}

            prefix_ids = self.tokenizer(prefix).input_ids
            labels_ids = tokenized['input_ids'].clone()
            labels_ids[:len(prefix_ids)] = -100 # Masks the prefix tokens in the labels with -100 for GPT loss computation.
            tokenized['labels'] = labels_ids
            tokenized['label_str'] = str(self.LABEL2ID[label])
            return tokenized
        else:
            text = f"Premise: {premise}\nHypothesis: {hypo}\nLabel:"
            tokenized = self.tokenizer(
                text,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt"
            )
            tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}
            tokenized['label_str'] = str(self.LABEL2ID[label])
            return tokenized

In [13]:
import time


# Load a pretrained GPT-2 model with official weights
model = GPT2Model.from_pretrained("gpt2").to(DEVICE)

# Then continue from checkpoint if training was disrupted
model_filenames = ["model_en_epoch_3", "model_vi_epoch_3", "model_zh_epoch_4", "model_es_epoch_4"]


# Testting per-language weights on XNLI dataset
for index, lang in enumerate (["en", "vi", "zh", "es"]):
    model.load_state_dict(torch.load(f"./best_model/{model_filenames[index]}.pt"))
    model.eval()
    
    test_dataset = XNLIDataset(
        split="test",
        lang=lang,
        tokenizer=tokenizer,
        subset=TEST_SUBSET
    )
    
    test_loader = DataLoader(test_dataset,shuffle=False,collate_fn=XNLIDataset.collate_fn)
    
    start_time = time.time()
    with torch.no_grad():
        acc, all_preds, all_labels = evaluate_gpt2_xnli(model, tokenizer, test_loader, max_gen_length=1, device=DEVICE)
    end_time = time.time()
    time_taken = end_time - start_time

    print(f"Training took {time_taken/60:.2f} minutes for all {len(test_dataset)} samples")




Dataset initialized: split='test', lang='en', total=5010, subset=1, subset_count=5010


Generating: 100%|██████████| 5010/5010 [00:45<00:00, 110.89it/s]


Evaluation accuracy: 83.23%
Training took 0.75 minutes for all 5010 samples
Dataset initialized: split='test', lang='vi', total=5010, subset=1, subset_count=5010


Generating: 100%|██████████| 5010/5010 [00:51<00:00, 97.38it/s] 


Evaluation accuracy: 65.67%
Training took 0.86 minutes for all 5010 samples
Dataset initialized: split='test', lang='zh', total=5010, subset=1, subset_count=5010


Generating: 100%|██████████| 5010/5010 [00:49<00:00, 100.52it/s]


Evaluation accuracy: 65.47%
Training took 0.83 minutes for all 5010 samples
Dataset initialized: split='test', lang='es', total=5010, subset=1, subset_count=5010


Generating: 100%|██████████| 5010/5010 [00:47<00:00, 104.85it/s]

Evaluation accuracy: 70.16%
Training took 0.80 minutes for all 5010 samples


In [15]:

# Test per-language weights on code-switched dataset
for lang in ["vi", "zh", "es"]:
    for filename in model_filenames:
        model.load_state_dict(torch.load(f"./best_model/{filename}.pt"))
        model.eval()
    
        test_dataset = CSDataset(
            split="test",
            lang=lang,
            tokenizer=tokenizer,
            subset=TEST_SUBSET
        )
                    
        test_loader = DataLoader(test_dataset,shuffle=False,collate_fn=XNLIDataset.collate_fn)

        start_time = time.time()
        with torch.no_grad():
            acc, all_preds, all_labels = evaluate_gpt2_xnli(model, tokenizer, test_loader, max_gen_length=1, device=DEVICE)
        end_time = time.time()
        time_taken = end_time - start_time

        print(f"Training took {time_taken/60:.2f} minutes for all {len(test_dataset)} samples")

Dataset initialized: split='test', lang='vi', total=1000, subset=1, subset_count=1000


Generating: 100%|██████████| 1000/1000 [00:09<00:00, 110.13it/s]


Evaluation accuracy: 68.30%
Training took 0.15 minutes for all 1000 samples
Dataset initialized: split='test', lang='vi', total=1000, subset=1, subset_count=1000


Generating: 100%|██████████| 1000/1000 [00:09<00:00, 107.61it/s]


Evaluation accuracy: 48.00%
Training took 0.15 minutes for all 1000 samples
Dataset initialized: split='test', lang='vi', total=1000, subset=1, subset_count=1000


...

Evaluation accuracy: 39.00%
Training took 0.15 minutes for all 1000 samples
Dataset initialized: split='test', lang='es', total=1000, subset=1, subset_count=1000


Generating: 100%|██████████| 1000/1000 [00:08<00:00, 111.72it/s]


Evaluation accuracy: 37.10%
Training took 0.15 minutes for all 1000 samples
Dataset initialized: split='test', lang='es', total=1000, subset=1, subset_count=1000


Generating: 100%|██████████| 1000/1000 [00:08<00:00, 113.68it/s]

Evaluation accuracy: 57.70%
Training took 0.15 minutes for all 1000 samples
